## Project Description

* Ingests and indexes multiple PDFs into a unified `FAISS vectorstore` with chunk-level source tagging

* Uses `HuggingFace MiniLM embeddings` and a custom score-threshold retriever to feed only high-confidence context to the LLM

* Leverages `Groq’s LLaMA-3 via LangChain’s ConversationalRetrievalChain` for memory-aware, back-and-forth document chat

* Prepends each answer with its `similarity score` and source filename, and falls back to “no reliable answer” when confidence is low

* Exports full chat history as organized `JSON` and `Markdown` logs under a dedicated chat_logs/ folder

* Modular, CLI-driven design ready for extension into web UIs, session management, or advanced debugging



## Install Dependencies

In [4]:
!pip install langchain faiss-cpu PyMuPDF sentence-transformers openai -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.93.0
    Uninstalling openai-1.93.0:
      Successfully uninstalled openai-1.93.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0


In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.3 MB/s eta 0:00:00


In [5]:
import os
from pathlib import Path
from typing import List
import fitz
import json
import datetime

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## 0 | Set API

In [ ]:
os.environ["OPENAI_API_KEY"] = "here goes nothing"
GROQ_API_BASE = "https://api.groq.com/openai/v1"
LLM_MODEL_NAME = "llama3-70b-8192"

## 1 | PDF Text Extraction

In [7]:
def extract_text_from_pdf(path: str) -> str:
    try:
        with fitz.open(path) as doc:
            return "\n".join(page.get_text() for page in doc)
    except Exception as e:
        raise RuntimeError(f"PDF extraction failed: {e}")

## 2 | Splitting Text to chunks

In [8]:
def split_text(text: str) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return splitter.split_documents([Document(page_content=text)])

## 3 | Load and Index PDFs

In [9]:
def load_and_index_pdfs(pdf_folder: str) -> FAISS:
    try:
        all_docs = []
        pdf_paths = list(Path(pdf_folder).glob("*.pdf"))

        for path in pdf_paths:
            raw_text = extract_text_from_pdf(str(path))
            chunks = split_text(raw_text)
            for chunk in chunks:
                chunk.metadata = {"source": path.name}
            all_docs.extend(chunks)

        print(f"✅ Loaded {len(pdf_paths)} PDFs with {len(all_docs)} total chunks")

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        return FAISS.from_documents(all_docs, embeddings)

    except Exception as e:
        raise RuntimeError(f"[ERROR] PDF batch processing failed: {e}")

## 4 | Custom Threshold Retriever

In [10]:
from langchain.schema import BaseRetriever
from pydantic import PrivateAttr

class ThresholdRetriever(BaseRetriever):
    _vectorstore: any = PrivateAttr()
    _threshold: float = PrivateAttr()
    _k: int = PrivateAttr()

    def __init__(self, vectorstore, threshold=0.75, k=4, **kwargs):
        super().__init__(**kwargs)
        self._vectorstore = vectorstore
        self._threshold = threshold
        self._k = k

    def get_relevant_documents(self, query):
        scored = self._vectorstore.similarity_search_with_score(query, k=self._k)
        return [doc for doc, score in scored if score >= self._threshold]

    async def aget_relevant_documents(self, query):
        return self.get_relevant_documents(query)

/tmp/ipython-input-10-402184082.py:4: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ThresholdRetriever(BaseRetriever):
/tmp/ipython-input-10-402184082.py:4: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class ThresholdRetriever(BaseRetriever):


## 5 | Build Chat Chain with Thresholding

In [11]:
# Build Chat Chain with Thresholding

def create_conversational_chain(vectorstore: FAISS) -> ConversationalRetrievalChain:
    llm = ChatOpenAI(
        model_name=LLM_MODEL_NAME,
        openai_api_base=GROQ_API_BASE,
        temperature=0.2
    )
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    retriever = ThresholdRetriever(vectorstore, threshold=0.75, k=5)

    return ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )

## 6 | Chat Loop with Anti-Hallucination Guard

In [12]:
# save the logs

def save_chat_logs(chat_log: list, filename_base: str, log_dir: str = "chat_logs"):
    os.makedirs(log_dir, exist_ok=True)

    json_path = os.path.join(log_dir, f"{filename_base}.json")
    md_path = os.path.join(log_dir, f"{filename_base}.md")

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(chat_log, f, indent=2, ensure_ascii=False)

    with open(md_path, "w", encoding="utf-8") as f:
        f.write("# Chat with Multiple PDFs\n\n")
        for turn in chat_log:
            f.write(f"**🧑 You:** {turn['user']}\n\n")
            f.write(f"**🤖 AI:** {turn['ai']}\n\n")

    print(f"\n💾 Chat logs saved to:\n - {json_path}\n - {md_path}")


In [13]:
def chat_with_pdfs(pdf_folder: str):
    vectorstore = load_and_index_pdfs(pdf_folder)
    convo_chain = create_conversational_chain(vectorstore)

    print("\n💬 You can now chat with the documents. Type 'exit' to quit.\n")
    chat_log = []
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename_base = f"chat_history_{timestamp}"

    while True:
        query = input("🧑 You: ").strip()
        if not query or query.lower() in ("exit", "quit"):
            print("👋 Ending chat.")
            break
        if len(query) < 3:
            print("⚠️  Enter a valid question.\n")
            continue

        response = convo_chain.invoke({"question": query})
        print(f"🤖 AI: {response['answer']}\n")

        chat_log.append({
            "timestamp": datetime.datetime.now().isoformat(),
            "user": query,
            "ai": response['answer']
        })

    # Save chat logs
    save_chat_logs(chat_log, filename_base)

In [16]:
chat_with_pdfs("/content/data")

✅ Loaded 2 PDFs with 88 total chunks

💬 You can now chat with the documents. Type 'exit' to quit.

🧑 You: what is the college that the student is studying in ?
🤖 AI: The college that the student, Sai Vidith Kota, is studying in is R.V.R & J.C College of Engineering, located in Chowdawaram, Guntur.

🧑 You: when will he graduate ?
🤖 AI: The expected graduation date of Sai Vidith Kota from R.V.R & J.C College of Engineering is September 2026.

🧑 You: exit
👋 Ending chat.

💾 Chat logs saved to:
 - chat_logs/chat_history_20250708_111040.json
 - chat_logs/chat_history_20250708_111040.md
